## Lecture Notes MSAN694

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

## Quiz 3: Covers Material from Week 2-3-4

Data - create the basic RDD and common python. Harder than last time, because more materials have been covered. Practice with the examples provided. 

## From Week 3:

### CombineByKey

Similiar to aggregate(), but also includes the kkey and the values.


**createCombiner** - initial value

**mergeValue** - if not initial, then aggregator Partition

**mergeCombiner** - this is adding across partitions

```python
'I love tacos'
'I love coffee'
'I love coffee'
```

**Target Output** 

```python
(len, (occurence, word))
```




** Step by Step **

```python

.textFile(' ')
.flatMap(lambda x ; x.split())
.map(lambda x: (len(x), x)

output -> partition 1: (1,I) (4,love) (5,tacos), (1,I) (4,love) (6,coffee)
output -> partition 2: (1,I) (4, love) (6, coffee)

# SAMPLE FUNCTIONS (WHATS HAPPENING IN THE BACKGROUND)
# will nest the tuple in another key
createCombiner: lambda x: (1,x)
mergevalue : lambda x,y : x[0] + 1, x[1] +', '+ y

(1,I) (4,love) (5,tacos), (1,I) (4,love) (6,coffee)

```
#### Partition 1:
```python
(5, (1, tacos))
(1, (2, I, I)
(4, (1+1, love, love))
(6, (1, coffee))
```
#### Partition 2:
```python
(1, (1, I))
(4, (1, love))
(6, (1, coffee))
```

#### How do you combine between partitions? 

```python
mergeCombiner: lambda x,y : x[0] + y[0], x[1] + "," + y[1]
```

#### All together

Before the data comes in, its already in (length, word) format
```python
(2, hi)
(4, love)
...
```

Then run this function

```python
comp = words.combineByKey(
                    (lambda x : (1,x)), 
                   (lambda x,y : (x[0] +1 , x[1] +', '+ y)), 
                   (lambda x,y : (x[0] + y[0], x[1] +', '+y[1])))
```

##### First it will nest the base value
```python
(key, value)
lambda x : (1,x)
(key, (occurance(default=1), growing list of words))

```


```python
(2 (1, hi))
```

##### Let's add (2,to), it has the same key so it will combine:

```python
(2 (1, hi))
lambda x,y : (x[0] +1 , x[1] +', '+ y)
(2 (2, hi, to))
```
##### Let's add (3,bye), it has the same key so it will combine:

```python
(2 (2, hi, to))
(3 (1, bye))
```

##### And so on




## Week 3: Example 7

In [6]:
lines = sc.textFile(text)
lines

I love tacos
I love coffee
I love coffee
 MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0

In [28]:
PATH = '/Users/tlee010/Desktop/github_repos/2017-msan694-example/Data/README.md'
lines = sc.textFile(PATH)

In [45]:
words = lines.flatMap(lambda x : x. split(' '))
words = words.map(lambda x : (len(x),x))

In [46]:
words.collect()[:9]

[(1, u'#'),
 (6, u'Apache'),
 (5, u'Spark'),
 (0, u''),
 (5, u'Spark'),
 (2, u'is'),
 (1, u'a'),
 (4, u'fast'),
 (3, u'and')]

In [48]:
comp = words.combineByKey(
                    (lambda x : (1,x)), 
                   (lambda x,y : (x[0] +1 , x[1] +', '+ y)), 
                   (lambda x,y : (x[0] + y[0], x[1] +', '+y[1])))

comp.collect()[:6]

[(0,
  (68,
   u', , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ')),
 (112,
  (1,
   u'[Eclipse](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools#UsefulDeveloperTools-Eclipse)')),
 (2,
  (70,
   u'is, It, in, R,, an, It, of, ##, on, ##, is, To, do, to, do, if, by, -T, in, is, at, an, ##, to, is, to, ##, if, ##, in, To, of, Pi, to, to, be, or, to, on, to, or, to, an, if, is, in, of, if, no, ##, is, be, on, to, or, ##, to, to, in, of, to, at, on, of, ##, to, in, an, on, to')),
 (4,
  (47,
   u'fast, APIs, that, data, also, rich, find, This, file, only, run:, (You, need, this, more, than, with, More, from, IDE,, also, also, with, will, when, This, URL,, with, with, also, name, Many, help, Once, [run, Note, uses, core, talk, HDFS, have, must, same, that, your, Hive, Hive')),
 (6,
  (41,
   u'Apache, system, Scala,, engine, graphs, GraphX, stream, Online, latest, guide,, README, th

### Question 8:

Which operations don't  shuffle data?


    - mapValues() <- O WITHIN the partition applies
    - groupByKey() <-- X will shuffle ( up front)
    - reduceByKey() <-- X will shuffle (at the end) 
    - combineBykey() <-- X will shuffle (at the end)
    - sortByKey() <-- X will shuffle
    - flatMapValues() <- O WITHIN the partition applies

## Week4 : Pair RDD

- Pair RDD
- Pair RDD Operations - Actions


### Actions 
- subtractByKey
- join
- rightOuterJoin
- leftOuterJoin

### Subtract by Key
- Only compares the keys

#### Example:

```python
A = [(2,3), (1,2), (1,3)]
B = [(1,2)]
```

#### Note that it ignores the actual values in the pairs, only compares the keys
```python
A.subtract(B) = [(2,3), (1,3)]
A.subtractByKey(B) = [(2,3)]
```

### Week4 : Example 1

List business names wihtout a district supervisor

In [79]:
PATH1 = '/Users/tlee010/Desktop/github_repos/2017-msan694-example/Data/supervisor_sf.csv'
PATH2 = '/Users/tlee010/Desktop/github_repos/2017-msan694-example/Data/filtered_registered_business_sf.csv'

In [80]:
lines1 = sc.textFile(PATH1)
lines2 = sc.textFile(PATH2)

In [86]:
# split by word
zippairs = lines1.map(lambda x : x.split(','))

# then turn into tuple 
zippairs = zippairs.map(lambda x : (x[0],x[1]))

# look at data
zippairs.take(5)

[(u'94102', u'8'),
 (u'94102', u'6'),
 (u'94102', u'3'),
 (u'94102', u'5'),
 (u'94103', u'8')]

In [87]:
# split the 2nd file by commas
zipbus = lines2.map(lambda x : x.split(','))

# split the 2nd file by commas
zipbus = zipbus.map(lambda x : (x[0], x[1]))
zipbus.take(5)

[(u'94123', u'Tournahu George L'),
 (u'94124', u'Stephens Institute Inc'),
 (u'94105', u'Stephens Institute Inc'),
 (u'94108', u'Stephens Institute Inc'),
 (u'94107', u'Stephens Institute Inc')]

In [88]:
res = zipbus.subtractByKey(zippairs).values().distinct()
res.take(5)

[u'Precision Communication Serv',
 u'Schefer Thomas R',
 u'Tyrrell Thomas',
 u'Lucid Systems',
 u'Jacob Abraham']

## Week 4 : Join , rightOuterjoin, LeftOuterjoin
## Week 4 : cogroup(otherdataset)

Group data from both RDDs sharing the same key. Go over two RDDs sharing the same key. Return the key and respective lists from two RDD values.

##### Group by Illustration
```python
.groupByKey()

a = [(1,2),(1,3)] -- > group by key --> [1, <iterable>] = [1, [2,3]]

b = [(2,2),(2,3)] --> group by key -- > [2, <iterable>] = [2, [2,3]]
```

##### Cogroup sample
```python
.cogroup(1 other dataset)

(1, (<iterable>,<iterable>)) = (1, ([2,3],[3]))

(2, (<iterable>,<iterable>)) = (2, ([],[2,3])) 

```

### Week4 : Example 2

Try out the join commands

In [92]:
first_num_pairs = sc.parallelize({(2,3), (1,2),(1,3),(2,4),(3,6)})

second_num_pairs = sc.parallelize({(1,3),(2,2)})

In [94]:
first_num_pairs.collect()

[(1, 2), (1, 3), (2, 3), (2, 4), (3, 6)]

In [95]:
second_num_pairs.collect()

[(1, 3), (2, 2)]

In [97]:
first_num_pairs.subtract(second_num_pairs).collect()

[(2, 3), (3, 6), (1, 2), (2, 4)]

In [98]:
first_num_pairs.subtractByKey(second_num_pairs).collect()

[(3, 6)]

In [99]:
first_num_pairs.join(second_num_pairs).collect()

[(1, (2, 3)), (1, (3, 3)), (2, (3, 2)), (2, (4, 2))]

### Note that its reversed, RIGHT = keep the originals, add additiona new data

In [100]:
first_num_pairs.rightOuterJoin(second_num_pairs).collect()

[(1, (2, 3)), (1, (3, 3)), (2, (3, 2)), (2, (4, 2))]

### Note that its reversed, LEFT = only return rows found in new dataset

In [101]:
first_num_pairs.leftOuterJoin(second_num_pairs).collect()

[(1, (2, 3)), (1, (3, 3)), (2, (3, 2)), (2, (4, 2)), (3, (6, None))]

In [102]:
first_num_pairs.cogroup(second_num_pairs).collect()

[(1,
  (<pyspark.resultiterable.ResultIterable at 0x117073790>,
   <pyspark.resultiterable.ResultIterable at 0x115c6bc50>)),
 (2,
  (<pyspark.resultiterable.ResultIterable at 0x115c6b850>,
   <pyspark.resultiterable.ResultIterable at 0x115c6b890>)),
 (3,
  (<pyspark.resultiterable.ResultIterable at 0x115c6b310>,
   <pyspark.resultiterable.ResultIterable at 0x115c6b050>))]

### Week4 : Example 3

Using Example 1 list (zip (business_name, supervisor_id) pairs ordered  y supervisor_id
- only if both business and supervisor exist
- if a business exists
- if a supervisor exists

In [103]:
lines1.take(5)

[u'94102,8', u'94102,6', u'94102,3', u'94102,5', u'94103,8']

In [104]:
lines2.take(5)

[u'94123,Tournahu George L,3301 Broderick St,San Francisco,CA',
 u'94124,Stephens Institute Inc,2225 Jerrold Ave,San Francisco,CA',
 u'94105,Stephens Institute Inc,180 New Montgomery St,San Francisco,CA',
 u'94108,Stephens Institute Inc,540 Powell St,San Francisco,CA',
 u'94107,Stephens Institute Inc,460 Townsend St,San Francisco,CA']

In [106]:
zipbus = lines2.map(lambda x : x.split(','))
zipbus = zipbus.map(lambda x : (x[0],x[1]))
zipbus.take(4)

[(u'94123', u'Tournahu George L'),
 (u'94124', u'Stephens Institute Inc'),
 (u'94105', u'Stephens Institute Inc'),
 (u'94108', u'Stephens Institute Inc')]

In [110]:
zips_super = lines1.map(lambda x : x.split(','))
zips_super = zips_super.map(lambda x : (x[0],x[1]))
zips_super.take(4)

[(u'94102', u'8'), (u'94102', u'6'), (u'94102', u'3'), (u'94102', u'5')]

### If both business and supervisor exist (inner join)

In [123]:
zip_comb = zipbus.join(zips_super).sortBy(lambda x : x[1][1])
zip_comb.take(5)

[(u'94117', (u'Alouis Auto Radiator Inc', u'1')),
 (u'94117', (u'Atlantic Richfield Co', u'1')),
 (u'94117', (u'Breall William Etal', u'1')),
 (u'94117', (u'Cole Hardware Inc', u'1')),
 (u'94117', (u'Cole Hardware Inc', u'1'))]

### If a business exists

In [126]:
zip_comb = zipbus.rightOuterJoin(zips_super).sortBy(lambda x : x[1][1])
zip_comb.take(5)

[(u'94117', (u'Alouis Auto Radiator Inc', u'1')),
 (u'94117', (u'Atlantic Richfield Co', u'1')),
 (u'94117', (u'Breall William Etal', u'1')),
 (u'94117', (u'Cole Hardware Inc', u'1')),
 (u'94117', (u'Cole Hardware Inc', u'1'))]

### If a supervisor exists

In [127]:
zip_comb = zipbus.leftOuterJoin(zips_super).sortBy(lambda x : x[1][1])
zip_comb.take(5)

[(u'', (u'Hartmann Studios Incorporated', None)),
 (u'', (u'Cardno Entrix', None)),
 (u'', (u'Bond Blacktop Inc', None)),
 (u'', (u'Moonka Nishi', None)),
 (u'', (u'Cooper Jim B', None))]

### Week4 : Example 4


Genearte Key(zip) and value pair RDDs from “filtered_registered_business_sf.csv” and “supervisor_sf.csv” and cogroup() the RDDs.

In [130]:
superzips = lines1.map(lambda x: x.split(','))
superzips = superzips.map(lambda x : (x[0],x[1]))
superzips.take(5)

[(u'94102', u'8'),
 (u'94102', u'6'),
 (u'94102', u'3'),
 (u'94102', u'5'),
 (u'94103', u'8')]

In [131]:
businesszips = lines2.map(lambda x: x.split(','))
businesszips = businesszips.map(lambda x : (x[0],x[1]))
businesszips.take(5)

[(u'94123', u'Tournahu George L'),
 (u'94124', u'Stephens Institute Inc'),
 (u'94105', u'Stephens Institute Inc'),
 (u'94108', u'Stephens Institute Inc'),
 (u'94107', u'Stephens Institute Inc')]

In [134]:
superzips.cogroup(businesszips).take(6)

[(u'',
  (<pyspark.resultiterable.ResultIterable at 0x1163a5ed0>,
   <pyspark.resultiterable.ResultIterable at 0x1163ab0d0>)),
 (u'85716',
  (<pyspark.resultiterable.ResultIterable at 0x1163ab310>,
   <pyspark.resultiterable.ResultIterable at 0x1163ab350>)),
 (u'28036',
  (<pyspark.resultiterable.ResultIterable at 0x1163ab390>,
   <pyspark.resultiterable.ResultIterable at 0x1163ab3d0>)),
 (u'97302',
  (<pyspark.resultiterable.ResultIterable at 0x1163ab410>,
   <pyspark.resultiterable.ResultIterable at 0x1163ab450>)),
 (u'55344',
  (<pyspark.resultiterable.ResultIterable at 0x1163ab510>,
   <pyspark.resultiterable.ResultIterable at 0x1163ab550>)),
 (u'94035',
  (<pyspark.resultiterable.ResultIterable at 0x1163ab590>,
   <pyspark.resultiterable.ResultIterable at 0x1163ab5d0>))]

In [136]:
superzips.countByKey()

defaultdict(int,
            {u'94102': 4,
             u'94103': 6,
             u'94104': 2,
             u'94105': 2,
             u'94107': 2,
             u'94108': 2,
             u'94109': 4,
             u'94110': 4,
             u'94111': 2,
             u'94112': 5,
             u'94114': 3,
             u'94115': 3,
             u'94116': 2,
             u'94117': 4,
             u'94118': 3,
             u'94121': 2,
             u'94122': 4,
             u'94123': 1,
             u'94124': 2,
             u'94127': 3,
             u'94129': 1,
             u'94130': 1,
             u'94131': 3,
             u'94132': 3,
             u'94133': 2,
             u'94134': 3,
             u'94158': 2})

In [137]:
superzips.lookup('94112')

[u'7', u'8', u'11', u'9', u'10']